In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
%pip install git+https://github.com/neelnanda-io/Easy-Transformer.git@clean-transformer-demo
#Install another version of node that makes PySvelte work way faster
!curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
%pip install git+https://github.com/neelnanda-io/PySvelte.git
%pip install fancy_einsum
%pip install einops

  Cloning https://github.com/neelnanda-io/Easy-Transformer.git (to revision clean-transformer-demo) to /tmp/pip-req-build-l5_56s1t
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/Easy-Transformer.git /tmp/pip-req-build-l5_56s1t
  Running command git checkout -b clean-transformer-demo --track origin/clean-transformer-demo
  Switched to a new branch 'clean-transformer-demo'
  Branch 'clean-transformer-demo' set up to track remote branch 'clean-transformer-demo' from 'origin'.
  Resolved https://github.com/neelnanda-io/Easy-Transformer.git to commit 1f25219e631aeb478d17075d47274db32c874e88
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/5

In [3]:
import einops
from fancy_einsum import einsum
from dataclasses import dataclass
from easy_transformer import EasyTransformer
import torch
import torch.nn as nn
import numpy as np
import math
from easy_transformer.utils import get_corner, gelu_new, tokenize_and_concatenate
import tqdm.auto as tqdm

In [4]:
reference_gpt2 = EasyTransformer.from_pretrained('gpt2-small',fold_ln=False, center_unembed=False, center_writing_weights=False)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

2025-04-21 20:28:32.848304: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745267313.098376      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745267313.171906      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Moving model to device:  cpu
Finished loading pretrained model gpt2-small into EasyTransformer!


In [5]:
sorted_vocab = sorted(list(reference_gpt2.tokenizer.vocab.items()), key=lambda n:n[1])
print(sorted_vocab[:20])
print()
print(sorted_vocab[250:270])
print()
print(sorted_vocab[990:1010])
print()

[('!', 0), ('"', 1), ('#', 2), ('$', 3), ('%', 4), ('&', 5), ("'", 6), ('(', 7), (')', 8), ('*', 9), ('+', 10), (',', 11), ('-', 12), ('.', 13), ('/', 14), ('0', 15), ('1', 16), ('2', 17), ('3', 18), ('4', 19)]

[('ľ', 250), ('Ŀ', 251), ('ŀ', 252), ('Ł', 253), ('ł', 254), ('Ń', 255), ('Ġt', 256), ('Ġa', 257), ('he', 258), ('in', 259), ('re', 260), ('on', 261), ('Ġthe', 262), ('er', 263), ('Ġs', 264), ('at', 265), ('Ġw', 266), ('Ġo', 267), ('en', 268), ('Ġc', 269)]

[('Ġprodu', 990), ('Ġstill', 991), ('led', 992), ('ah', 993), ('Ġhere', 994), ('Ġworld', 995), ('Ġthough', 996), ('Ġnum', 997), ('arch', 998), ('imes', 999), ('ale', 1000), ('ĠSe', 1001), ('ĠIf', 1002), ('//', 1003), ('ĠLe', 1004), ('Ġret', 1005), ('Ġref', 1006), ('Ġtrans', 1007), ('ner', 1008), ('ution', 1009)]



In [6]:
reference_text = "Mechanistic Interpretability aims to reverse engineer the reverse engineer the neural network."
tokens = reference_gpt2.to_tokens(reference_text)
print(tokens)
print(tokens.shape)
print(reference_gpt2.to_str_tokens(tokens))

tensor([[50256, 28452,   272,  2569, 48907,  1799, 12031,   284,  9575, 11949,
           262,  9575, 11949,   262, 17019,  3127,    13]])
torch.Size([1, 17])
['<|endoftext|>', 'Mech', 'an', 'istic', ' Interpret', 'ability', ' aims', ' to', ' reverse', ' engineer', ' the', ' reverse', ' engineer', ' the', ' neural', ' network', '.']


In [7]:
#tokens = tokens.cuda()
logits, cache = reference_gpt2.run_with_cache(tokens)
print(logits.shape)

torch.Size([1, 17, 50257])


In [8]:
log_probs = logits.log_softmax(dim=-1)
probs = logits.log_softmax(dim=-1)
print(log_probs.shape)
print(probs.shape)

torch.Size([1, 17, 50257])
torch.Size([1, 17, 50257])


In [9]:
list(zip(reference_gpt2.to_str_tokens(reference_text), reference_gpt2.tokenizer.batch_decode(logits.argmax(dim=-1)[0])))

[('<|endoftext|>', '\n'),
 ('Mech', 'anical'),
 ('an', 'ics'),
 ('istic', ' and'),
 (' Interpret', 'ation'),
 ('ability', ' of'),
 (' aims', ' to'),
 (' to', ' provide'),
 (' reverse', ' the'),
 (' engineer', ' the'),
 (' the', ' way'),
 (' reverse', ' engineering'),
 (' engineer', "'s"),
 (' the', ' way'),
 (' neural', ' network'),
 (' network', '.'),
 ('.', ' The')]

In [10]:
print(reference_gpt2.cfg)

EasyTransformerConfig(n_layers=12, d_model=768, n_ctx=1024, d_head=64, model_name='gpt2-small', n_heads=12, d_mlp=3072, act_fn='gelu_new', d_vocab=50257, eps=1e-05, use_attn_result=False, use_attn_scale=True, use_local_attn=False, model_family='gpt2', checkpoint=None, tokenizer_name='gpt2', window_size=None, attn_types=None, init_mode='gpt2', normalization_type='LN', device='cpu', attention_dir='causal', attn_only=False, seed=42, initializer_range=0.02886751345948129, init_weights=False, scale_attn_by_inverse_layer_idx=False, positional_embedding_type='standard', final_rms=False, d_vocab_out=50257, parallel_attn_mlp=False, rotary_dim=64, dtype=torch.float32)


In [11]:
import pysvelte
pysvelte.AttentionMulti(tokens=reference_gpt2.to_str_tokens(reference_text),attention=cache['blocks.0.attn.hook_attn'][0].permute(1,2,0)).show()

pysvelte components appear to be unbuilt or stale
Running npm install...

added 642 packages, and audited 643 packages in 13s

12 packages are looking for funding
  run `npm fund` for details

46 vulnerabilities (3 low, 5 moderate, 33 high, 5 critical)

To address issues that do not require attention, run:
  npm audit fix

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
Building pysvelte components with webpack...


npm notice 
npm notice New major version of npm available! 8.19.4 -> 11.3.0
npm notice Changelog: <https://github.com/npm/cli/releases/tag/v11.3.0>
npm notice Run `npm install -g npm@11.3.0` to update!
npm notice 



> pysvelte@1.0.0 webpack
> node ./node_modules/.bin/webpack

entry: {"loader":"./src/loader.js","AttentionMulti":"./src/AttentionMulti.svelte","Hello":"./src/Hello.svelte","TextSingle":"./src/TextSingle.svelte","TopKTable":"./src/TopKTable.svelte"}
asset AttentionMulti.js 40.2 KiB [emitted] [minimized] (name: AttentionMulti)
asset TopKTable.js 32.2 KiB [emitted] [minimized] (name: TopKTable)
asset loader.js 32.1 KiB [emitted] [minimized] (name: loader)
asset TextSingle.js 23.7 KiB [emitted] [minimized] (name: TextSingle)
asset Hello.js 19.9 KiB [emitted] [minimized] (name: Hello)
runtime modules 2.18 KiB 11 modules
orphan modules 72.9 KiB [orphan] 6 modules
cacheable modules 437 KiB
  modules by path ./node_modules/ 116 KiB
    modules by path ./node_modules/pako/lib/ 102 KiB 12 modules
    ./node_modules/numpy-parser/dist/main.js 3.63 KiB [built] [code generated]
    ./node_modules/ndarray/ndarray.js 9.62 KiB [built] [code generated]
    ./node_modules/iota-array/iota.js 150 bytes [b